# From Autoencoder to Variational Autoencoer and beyond

## Autoencoder

Before taking a deeper look into variational autoencoders we will briefly discuss simple autoencoders. An autoencoder is a special type of neural network which goal is to learn a latent representation of unlabeled data in a lower dimension. The learned encodings can therefore be viewed as a compression of the original data. An autoencoder consists of an encoder and a decoder part both implemented as a neural network. The encoder “tries” to find a good representation of the original data in lower dimension and the decoder “tries” to reconstruct the original data using this representation.  

<IMG SRC="assets/lena.PNG">
    <center>
        <a href="http://users.cecs.anu.edu.au/~Tom.Gedeon/conf/ABCs2018/paper/ABCs2018_paper_58.pdf">image source</a>
    </center>

The whole idea of this kind of architecture is to introduce a bottleneck layer where only the structured part of the information can pass. Let us assume as an example the autoencoder is trained on faces. For a human an ideal autoencoder would learn descriptive attributes of faces such as skin color, smiling, wearing a hat etc allowing us to tweak those attributes easily. Another example is using an autoencoder for removing noises in a picture since noises are usually unstructured information. The bottleneck plays a key role when designing the neural network; the dimension of the latent space has to be carefully controlled and adjusted depending on the final purpose. If we pick the latent dimension too large the neural network won't learn any feature and will just memorize the input data whereas a too small latent dimension will drop loose too much information.

<IMG SRC="assets/lena_attributes.PNG">

The autoencoder is trained by comparing the original data with the output of the autoencoder (reconstructed data) and backpropagate the error to update the weights of the neural network. The loss function is therefore given by:

<math>
    \begin{align}
        Loss = \parallel x-\hat{x}\parallel_2 = x-d(z)=x-d(e(x)),
    \end{align}
<math>

where denotes $x$ the input data, $\hat{x}$ the recontructed data and $z$ the latent representation. Encoder and decoder are denoted by $e$ resp. $d$.

## Variational Autoencoder (High level view)
### Limitations of Autoencoders

Once we have trained an autoencoder the encoder can be used to encode the original data and the decoder can reconstruct it. But suppose we want to create a new picture (we will use pictures as example here but the same concept also applies to other kind of datas like audio or text) which has never been seen before. We want to sample a point from the latent space and use the decoder as an generator to create a new picture. This approach only works under the assumption that the latent space is well "organized" (e.g. following approximately a normal distribution), though. However, the organization of the latent space depends on the initial data (and the architecutre of the encoder) and the encoder has "no incentive" to organise the latent space (z-space) in a nice way so that we can sample from it easily since the encoder only tries to minimize the reconstruction loss. So, in order to be able to use the decoder of our autoencoder for generative purpose, we have to make sure that the latent space (z-space) is organized.
<p>Another issue with autoencoder is that a simple autoencoder tends to overfit the training data since the latent space can take any shape. <p>

<IMG SRC="assets/latent.PNG">
    <center> Note: For an autoencoder the z-space doesn't need to be connected and in practice it usually isn't </center>

### From Autoencoder to Variational Autoencoder

A VAE can also be viewed as a modification of a simple autoencoder without the mentioned limitations; hence we can sample from the latent space. We can add regularization by encoding an input as a distribution over the latent space instead of a single point. When reconstructing the input we just sample a single point from this distribution and use the decoder for reconstruction. This latent distribution depends on $x$ and it is denoted as $p(z \mid x)$. 

<IMG SRC="assets/compare.PNG">
        <center>
        <a href="https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73">image source</a>
    </center>

The main property we impose when encoding an input as a distribution is continuity which enables sampling from this distribution. Continuity means that there are no "gaps" between two different encodings. Or in other words: When drawing a line connecting two points in the latent space, every point on this line should also be decoded to something meaningful. Note that we here also impose that two inputs which latent representations are close to each other are also similar and when interpolating between two points in the latent space should produce a meaningful image.

<IMG SRC="assets/latent_gap1.PNG">
        <center>
        <a href="https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf">image source</a>
    </center>
    

The latent distribution $p(z)$ is called prior and in practice it is chosen to be standardnormal as it has many useful properties (later). Aside from minimizing the reconstruction loss, we also want the distribution $p(z \mid x)$ (posterior) to be close to the standardnormal distribution $p(z)$. We can enforce the distribution p(z) by adding an additional regularization term to the reconstruction loss. When comparing the similarity of two distributions $P$ and $Q$ a common measure is used: The Kullback-Leibler divergence which is defined as 

<math>
\begin{align}
D_{KL}(P \parallel Q) 
    &= -\sum_z P(z) \log \frac{P(z)}{Q(z)} \\
\end{align}
</math>

There is a deeper probablistic explanation why the Kullback-Leibler divergence is used as a regularization term (see next section). To further simplify calculations we also assume $p(z \mid x)$ belongs to the family of normal distributions which we will denote by $q(z \mid x)$ (for an abritary distribution $p(z \mid x)$ the computations would be intractable). We will see later that the Kullback-Leibler divergence between two Gaussian distributions has a closed form that can be expressed in terms of the means and the covariance matrices of the the two distributions. The loss function of an VAE is now given by:

<math>
\begin{align}
Loss = \parallel x-\hat{x}\parallel _2 + D_{KL}(q(z | x) \parallel p(z) ) 
\end{align}
</math>

As one can suspect, including a regularisation term will lead to an increase of the reconstruction error on the training data since we have two objective now. However, the tradeoff between the reconstruction error and the KL-loss can be monitored by scaling it down.

To summarize, the VAE can be derived from a simple autoencoder by encoding distributions instead of single points. We then add a regularization term to the loss function to make sure the latent distribution follows a standard normal distribution. The encoder is tweaked to output a distribution instead of a simple vector but the decoder remains the same.

## Probablistic view
Our main goal is beeing able to sample a representation and generate a new picture. Let us assume we have a hidden variable z which is not directly obeserved and generates an observation x. We can define our generative process by first sampling a latent representation z from p(z) which is also called prior distribution. Given this latent representation z we sample a new x from the conditional likelihood distribution $p(x | z)$. Since we don't directly observe the latent variable z our goal is to infer the characterisitcs of z given the observed data x, namely the distribution $p(z | x)$. Using Bayes theorem we get the following formula:

<math>
    \begin{align}
    p\left( {z|x} \right) = \frac{{p\left( {x|z} \right)p\left( z \right)}}{{p\left( x \right)}}= \frac{{p\left( {x|z} \right)p\left( z \right)}}{{\int {p\left( {x|y} \right)p\left( y \right)dy}}} 
    \end{align}
<math>

Unfortunately, the integral in the denominator requires exponential time to compute for a high dimensional latent space (we have to solve many integrals). A way to overcome computational difficulties is to use Variational Inference methods (hence the name Variational Autoencoder) to approximate the posterior distribution $p(z | x)$ with (tracable) distributions $q\left( z | x \right)$. Here $q\left( z |x \right)$ is chosen to be a Gaussian distribution and our goal is to find a Gaussian distribution that is most similar to the posterior. Note that this is euqivalent to finding $\mu$ and $\sigma$ so that it is most similar to the posterior whereby $\mu$ and $\sigma$ is depending of $x$. 

So we want our variational posterior $q(z | x)$ to be as close as possible to our true posterior $p(z|x)$. Recall that we can compare the similarity of two distributions by calculating the Kullback-Leibler divergence. In the end this is a minimization problem in the parameter $\lambda=(\mu,\sigma)$.

<math>
    \begin{align}
 D_{KL}\left( {q_\lambda\left( {z|x} \right)||p\left( {z|x} \right)} \right)= E_q(\log q_\lambda(z | x)) - E_q(\log p(x,z) + log p(x))
    \end{align}
<math>

As already mentioned, the computation of $p(x)$ is intractable so we have to reformulate our problem. Let us consider the Kullback-Leibler divergence of the distributions $q(z)$ and $p(z | x)$. The expression can rewritten as follows:

<math>
\begin{align}
D_{KL}(q \parallel p(z|x)) 
    &= -\sum_z q(z) \log \frac{p(z | x)}{q(z)} \\
    &= -\sum_z q(z) \log \frac{p(x,z)}{p(x) q(z)} \\
    &= -\sum_z q(z) \left[ \log \frac{p(x,z)}{q(z)} - \log p(x) \right]\\
    &= -\sum_z q(z) \log \frac{p(x,z)}{q(z)} + \sum_z q(z) \log p(x) \\
    &= -\sum_z q(z) \log \frac{p(x,z)}{q(z)} + \log p(x) \\\\
\log p(x) - D_{KL}(q \parallel p(z|x)) &= \sum_z q(z) \log \frac{p(x,z)}{q(z)}:=\mathbf{ELBO(\lambda)}
\end{align}
</math>

The right side term is called Evidence lower bound (ELBO) since KL is always positive and therefore ELBO<= log p(x). So we actually maximize the ELBO instead of minimizing the KL-Divergence of q(z) and p(z|x) which turned out to be equivalent since log p(x) is fixed (x is observed). Since we are not intergrating over p(x) this is tractable. After some transformation we get the loss function of a VAE introduced in the last section. Note that we here replace $q(z)$ with $q(z |x)$ to indicate the dependency on $x$.

<math>
\begin{align}
\sum_z q(z | x) \log \frac{p(x,z)}{q(z |x)} 
    &= \sum_z q(z | x) \log \frac{p(x | z) p(z)}{q(z | x)}  \\
    &= \sum_z q(z | x) \left[ \log p(x | z) + \log \frac{p(z)}{q(z | x)}  \right]\\
    &= \sum_z q(z | x) \log p(x | z) + \sum_z q(z |x) \log \frac{p(z)}{q(z | x)}\\
    &= E(\log p(x | z)) - D_{KL}(q(z | x) \parallel p(z) ) \\
\end{align}
</math>

Note that the first term actually is the reconstruction loss $\parallel x-\hat{x} \parallel_2^2$ when assuming the likelihood $p(x | z)$ to be Gaussian since the density function of a Gaussian takes the form of $Ce^{(...)^2}$ assuming unit covariance. Recall that we set p(z) to be Standard Gaussian with unit covariance. Hence, we also want q(z|X) to be as close as possible to a Standard Gaussian so that we could sample from it easily. Having $p(z) \sim \mathcal{N}(0,1)$ also adds another benefit. Since we have chosen the distribution q(z|x) to be Gaussian with parameters $μ_q$ and $Σ_q$ (we will write $μ_q$ and $Σ_q$ instead of $μ_q(x)$ and $Σ_q(x)$ to simplify notations), the KL divergence between those two distribution could be computed in closed form! (see appendix for calculations)

<math>
\begin{align}
D_{KL}(q||p) = \frac{1}{2}\left[\log\frac{|\Sigma_p|}{|\Sigma_q|} - k + (\mu_q-\mu_p)^T\Sigma_p^{-1}(\mu_q-\mu_p) + tr\left\{\Sigma_p^{-1}\Sigma_q\right\}\right]
\end{align}
</math>

Here k denotes the dimension of the latent space, $|\Sigma_q|$ is the determinant of the matrix $\Sigma_q$ and $tr(x)$ is the trace function. When p is standard normal distributed we have $\mu_p = 0$ and $\Sigma_p $ diagonal with entry 1 (unit covariance). Therefore, this term can be simplified to:

<math>
    \begin{align}
D_{KL}(q||p) &= \frac{1}{2} \left[ -\log|\Sigma_q|-k+\mu_q^T\mu_q + tr\left\{\Sigma_q \right\} \right]
\end{align}
</math>

Since $\Sigma_q$ is diagonal matrix we threat it as a vector of its diagonal entries to furhter simplify notations. We can rewrite the right hand side as:

<math>
    \begin{align}
&= \frac{1}{2} \, \left( \sum^k_{j=1} \Sigma_{q,j} + \sum^k_{j=1} \mu^2_{q,j} - \sum^k_{j=1} 1 - \log \, \prod^k_{j=1} \Sigma_{q,j} \right) \\
&= \frac{1}{2} \, \left( \sum^k_{j=1} \Sigma_{q,j} + \sum^k_{j=1} \mu^2_{q,j} - \sum^k_{j=1} 1 - \sum^k_{j=1} \log \Sigma_{q,j} \right) \\    
&= \frac{1}{2} \sum^k_{j=1} \left( \Sigma_{q,j} + \mu^2_{q,k} - 1 - \log \Sigma_{q,j} \right) 
\end{align}
</math>

$\Sigma_q$ is modeled as $log \Sigma_q$ in practice due to numerically stability and hence we end up with the KL-loss usually seen in implementations


<math>
\begin{align}
\frac{1}{2} \sum^k_{j=1} \left( \exp(\Sigma_{q,j}) + \mu^2_{q,j} - 1 - \Sigma_{q,j} \right) 
\end{align}
</math>

As already mentioned, the decoder and encoder of a VAE output a distribution instead of a vector given an input x. We are approximating the posterior $q(z | x)$ with a neural network (encoder) which output are the parameters of the distribution $(\lambda =(\mu,\sigma))$ in case of a normal distribution). The same holds for the likelihood $p(x |z) $ which is approximated by a generative network (decoder). The parameters (weights and biases) of the encoder and decoder are learnt by maximazing the ELBO loss function we derived.

## Vector Quantized Variational Autoencoder (VQ-VAE)

We have seen that by simple adding a regularization term allow us to generate new images. Compared to GANs (Generative adversial networks) they are also easier to train and reason about. A major drawback of VAEs is the unrealistic, blurry outputs they produce, though. The reason lies in the variational Bayes approach itself e.g. assuming a Gaussian output and and a continous latent space by adding a regularization term to the loss function. In other words, VAEs try to interpolate between real world objects but real objects are discrete and interpolating between them usually won't make sense.
Especially, when sampling from a multi-modal distribution (multiple classes), VAEs won't be able to generate images with sharp edges and fine details. For example, a point in the latent space which lies between a point representing a dog and a point representing a car can't be decoded to a meaningful image so the VAE "solve" this problem by blurring the images.

<IMG SRC="assets/vq.PNG">
    <center> Note: In reality this is much more complex and we don't have one vector representing one object. Additionally, the same vector at a different location in the feature map could produce a completely different image. </center>

A Vector Quantized Variational Autoencoder (VQ-VAE) solve this problem by discretizing the latent space. Instead of having one continous latent vector multiple discrete vectors will serve as input for the decoder to generate new images. One should keep in mind that there are only K different vectors you can choose from else this wouldn't be much different than a VAE.

## Intuition

The architecture of a VQ-VAE looks similar to the architecture of an autoencoder except of an additional codebook consisting k different 1-dimensional vectors of a predefined length. The purpose of the codeboo is to quantize the output of the encoder. In contrast to an autoencoder we won't flatten the output of the encoder and take the 3-dimensional tensor (green), the representation of the image, for the quantization step. The codebook contains k vectors $e_{1,...,k}$of dimension D which must correspond to the channels of the output of the encoder. In the quantization step each vector of length D (red) of the output of the encoder $z_e(x)$ (green) will be replaced by the vector in the codebook which is closest (euclidian distance) to obtain $z_q(x)$ (violet). Eventually, this will be used as input to reconstruct the original image.

<IMG SRC="assets/vqvae.PNG">
    <center>
        <a href="https://arxiv.org/pdf/1711.00937.pdf">image source</a>

Mathematically, we can write
<math>
\begin{align}
    z_q(x)=e_k, \quad \text{where} \quad k = argmin_j \parallel z_e(x) − e_j \parallel_2
\end{align}
</math>

The posterior categorical distribution q(z|x) probabilities are defined as one-hot:
<math>
\begin{align}
q(z = k|x) = \left\{
\begin{array}{ll}
1 & k = \textrm{for} \quad argmin_j \parallel z_e(x) − ej_k \parallel_2, \\
0 & \, \textrm{otherwise} \\
\end{array}
\right. 
\end{align}
</math>

One should note that limiting the amount of possible codebook vectors to K isn't as limiting as it seems to be. For instance, if the last layer of the encoder outputs a $32x32x128$ tensor and the coodebook consists of $K=64$ vectors (of dimension 128) the decoder is already able to generate $64^{32x32} \approx 3.3*10^{1849}$ (this is a very big number) different images. There's just one caveat when we want to backpropagate the loss: The gradient of the "replace with codebook vctor" operation (argmin function) doesn't exist. This is solved by approximating the gradient with the gradient of the decoder (red arrow). The reasoning of this approximation is that when $z_q(x)$ and $z_e(x)$ are close to each other the gradient may also pointing in the same direction approximately (see figure). 

### Loss function
Apart from the weights of the encoder and decoder the codebook vectors also have to be learnt. Ideally our encoder will output vectors that are close to one of the learned codebook vectors. There is essentially a bi-directional problem here: One the one hand we want to push the learned codebook vectors as close to the encoder outputs and on the other hand we want  the encoder output vectors to be as close as possible to one of the codebook vector.
Both of these problems are solved by adding two additional terms to the reconsctruction loss function

<math>
\begin{align}
    Loss=log(p(x∣q(x))) + \parallel sg[z_e(x)]−e \parallel^2_2 + β \parallel z_e(x)−sg[e] \parallel^2_2
\end{align}
</math>

$\text{sg}[x]$ stands for “stop gradient” and prevents an update of the learnable variables/weights of that part of the equation. The second term's goal is to push the codebook vectors as close as possible to their corresponding encoded vectors when the encoded vectors are frozen. The third term is called commitment loss and works the other way around. The codebook vectors are frozen and the encoder outputs are pushed as close as possible to their closest codebook vector. The importance of this term can be tuned by the hyperparameter $\beta$. Note that the decoder optimises the first loss term only, the encoder optimises the first and the last loss terms, and the embeddings are optimised by the middle loss term.

In the VAE framework we enforced a static prior p(z) on the latent space. 
During training, VQ-VAE assumes a uniform prior for z, so all latents codes are equally likely (and appear with a probability of $1/k$). Since the posterior q(z|x) is a unit function and the prior p(z) a uniform distribution the KL term that appears in the ELBO end up to be constant and can be neglected.

<math>
\begin{align}
D_{KL}(p \parallel q) 
    &= \sum_z q(z) \log \frac{q(z)}{p(z)} = 1 * log (\frac{1}{\frac{1}{k}})=log(k)\\
\end{align}
</math>

### Sampling
What is left is how to sample from a learned network and codebook. After training we (freeze the networks weights and codebook and) fit an autoregressive distribution over z. By denoting $z_i$ as the i-th codebook vector we can write the autoregressive factorization as: 

<math>
    \begin{align}
p(z)=p(z_1)p(z_2|z_1)p(z3|z_1,z_2)...
        \end{align}
    </math>

This is similar to the case of language modeling where we try to predict the next word or subword given the previous subwords. We can use any autoregressive model to train the prior by backpropagating the cross-entropy loss. For images we can first flatten q(z|x) and threat it like a 1-dimensional sequence during training. When we want to sample new images we just feed in a start token and let the autoregressive model predict the codebook vectors which will be used as an input for the decoder of the VQ-VAE. In other words, an image is generated on a pixel-by-pixel basis.

<IMG SRC="assets/autoreg.PNG">

## Other ressources and references
### VAE:
<ol>
    <li><a href="https://arxiv.org/pdf/1906.02691.pdf">An Introduction to Variational Autoencoders</a> by Diederik P. Kingma and Max Welling</li>
    <li><a href="https://keras.io/examples/generative/vae/">Variational AutoEncoder</a> by François Chollet</li>
  <li><a href="https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73">Understanding  Variational Autoencoders (VAE)</a> by Joseph Rocca</li>
  <li><a href="https://jaan.io/what-is-variational-autoencoder-vae-tutorial/">Tutorial - What is a variational autoencoder?</a> by Jaan Altosaar</li>
   <li><a href="https://www.jeremyjordan.me/variational-autoencoders/">Variational autoencoders</a> by Jeremy Jordan</li>     
</ol>

### VQ-VAE:
<ol>
    <li><a href="https://arxiv.org/abs/1711.00937">Neural Discrete Representation Learning</a> by Aaron van den Oord et al.</li>
  <li><a href="https://keras.io/examples/generative/vq_vae/">Vector-Quantized Variational Autoencoders</a> by Sayak Paul</li>
  <li><a href="https://ml.berkeley.edu/blog/posts/vq-vae/">Understanding VQ-VAE (DALL-E Explained Pt. 1)</a> by Charlie Snell</li>

</ol> 

## Appendix

Adjusted version of: https://mr-easy.github.io/2020-04-16-kl-divergence-between-2-gaussian-distributions/

KL divergence between two distributions 
P and Q of a continuous random variable is given by:

<math>
\begin{align}
D_{KL}(p||q) = \int_x p(x) \log \frac{p(x)}{q(x)}
\end{align}
</math>

And probabilty density function of multivariate Normal distribution is given by:

<math>
\begin{align}
p(\mathbf{x}) = \frac{1}{(2\pi)^{k/2}|\Sigma|^{1/2}} \exp\left(-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^T\Sigma^{-1}(\mathbf{x}-\boldsymbol{\mu})\right)
\end{align}
</math>

Let our two Normal distributions be $\mathcal{N}(\boldsymbol{\mu_p},\,\Sigma_p)$ and $\mathcal{N}(\boldsymbol{\mu_q},\,\Sigma_q)$, both k dimensional.

<math>
\begin{aligned}
D_{KL}(p||q) & = \mathbb{E}_p\left[\log(p) - \log(q)\right]
\newline
& = \mathbb{E}_p\left[\frac{1}{2}\log\frac{|\Sigma_q|}{|\Sigma_p|} - \frac{1}{2}(\mathbf{x}-\boldsymbol{\mu_p})^T\Sigma_p^{-1}(\mathbf{x}-\boldsymbol{\mu_p}) + \frac{1}{2}(\mathbf{x}-\boldsymbol{\mu_q})^T\Sigma_q^{-1}(\mathbf{x}-\boldsymbol{\mu_q})\right]
\newline
& = \frac{1}{2}\mathbb{E}_p\left[\log\frac{|\Sigma_q|}{|\Sigma_p|}\right] - \frac{1}{2}\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_p})^T\Sigma_p^{-1}(\mathbf{x}-\boldsymbol{\mu_p})\right] + \frac{1}{2}\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_q})^T\Sigma_q^{-1}(\mathbf{x}-\boldsymbol{\mu_q})\right]
\newline
& = \frac{1}{2}\log\frac{|\Sigma_q|}{|\Sigma_p|} - 
\underbrace{\frac{1}{2}\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_p})^T\Sigma_p^{-1}(\mathbf{x}-\boldsymbol{\mu_p})\right]}_{\substack{\boldsymbol{:=\, L2}}} + 
\underbrace{\frac{1}{2}\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_q})^T\Sigma_q^{-1}(\mathbf{x}-\boldsymbol{\mu_q})\right]}_{\substack{\boldsymbol{:=\, L3}}}
\end{aligned}
</math>

 Using the cyclic property of trace: $tr(ABC) = tr(BCA) = tr(CAB)$ and $\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_p})(\mathbf{x}-\boldsymbol{\mu_p})^T\right] = \Sigma_p$ the second simplifies to

<math>
\begin{align}
L2 &= \frac{1}{2}\mathbb{E}_p\left[tr\left\{(\mathbf{x}-\boldsymbol{\mu_p})(\mathbf{x}-\boldsymbol{\mu_p})^T\Sigma_p^{-1}\right\}\right] \newline
& = \frac{1}{2}tr\left\{\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_p})(\mathbf{x}-\boldsymbol{\mu_p})^T\Sigma_p^{-1}\right]\right\} \newline
    & = \frac{1}{2}tr\left\{\mathbb{E}_p\left[(\mathbf{x}-\boldsymbol{\mu_p})(\mathbf{x}-\boldsymbol{\mu_p})^T\right]\Sigma_p^{-1}\right\}\newline
    & = \frac{1}{2}tr\left\{\Sigma_p\Sigma_p^{-1}\right\}
\newline
& = \frac{1}{2}tr\left\{I_k\right\}
\newline
& = \frac{k}{2}
\end{align}
</math>

Using the property: $E[(x − m')^T A(x − m')] = (m − m')^T A(m − m') + tr(A\Sigma)$ where $x \sim \mathcal{N}(m,\Sigma)$ the third term can be written as

<math>
\begin{align}
L3 = 
(\boldsymbol{\mu_p}-\boldsymbol{\mu_q})^T\Sigma_q^{-1}(\boldsymbol{\mu_p}-\boldsymbol{\mu_q}) + 
tr\left\{\Sigma_q^{-1}\Sigma_p\right\}
\end{align}
</math>

Putting everything together we get

<math>
\begin{align}
D_{KL}(p||q) = \frac{1}{2}\left[\log\frac{|\Sigma_q|}{|\Sigma_p|} - k + (\boldsymbol{\mu_p}-\boldsymbol{\mu_q})^T\Sigma_q^{-1}(\boldsymbol{\mu_p}-\boldsymbol{\mu_q}) + tr\left\{\Sigma_q^{-1}\Sigma_p\right\}\right]
\end{align}
</math>